In [1]:
import pandas as pd
import numpy as np

In [2]:
fn_df = pd.read_csv('FakeReal_News_Sites.csv', index_col=[0])
fn_df.head()

,Domain,Page rank integer,Page rank decimal,Site Rank,Domain registrar,Postal code,Country of origin,Harmless,Malicious,Suspicious,Undetected,Timeout,No. of Local links,No. of Outbound links,Total links,Links to fake news site?,Wordpress?,Site Rating
0,actionnews3.com,3,2.82,7390112.0,"GoDaddy.com, LLC",85284,US,86,0,0,10,0,16,1,17,False,True,Fake
1,abcnews-us.com,3,2.89,6579810.0,NamePal.com #8009 Inc.,32256,US,86,0,0,10,0,0,1,1,False,False,Fake
2,21stcenturywire.com,5,4.57,66742.0,"GoDaddy.com, LLC",85284,US,83,0,0,13,0,183,36,219,False,True,Fake
3,100percentfedup.com,4,4.15,778723.0,"GoDaddy.com, LLC",85284,US,84,1,0,11,0,49,8,57,False,True,Fake
4,abcnews.com,5,4.78,32787.0,"CSC CORPORATE DOMAINS, INC.",10023-6298,US,83,0,0,13,0,0,144,144,False,False,Fake


In [3]:
fn_df[(fn_df['Site Rating'] == 'Fake')].shape[0]

149

In [4]:
fn_df.loc[fn_df['Malicious'] > 0, 'Malicious'] = 1

In [5]:
#unnecessary columns = Links to fake news site?, Suspicious, and No. of Outbound links
X = fn_df[['Domain registrar', 'Postal code',
       'Wordpress?', 'Page rank decimal', 'Site Rank', 'Country of origin',
       'Malicious', 'No. of Local links', 'Total links']]

y = fn_df['Site Rating']


In [7]:
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.9 MB/s eta 0:00:0000:0100:01


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

xgb = XGBClassifier()

le = LabelEncoder()
y = le.fit_transform(y)

cat_cols = ['Postal code', 'Country of origin']
bin_cols = ['Wordpress?', 'Malicious']
num_cols = ['Page rank decimal','Site Rank', 'Total links']

ct = ColumnTransformer([('standard_scaler', StandardScaler(), num_cols),
                        ('one_hot_encoder', OneHotEncoder(sparse=False, handle_unknown= "ignore"), cat_cols),
                        ('label_encoder', OrdinalEncoder(), bin_cols)])

clf = Pipeline(steps = [('preprocessor', ct), ('XGBoost', xgb)])



ModuleNotFoundError: No module named 'xgboost'

In [ ]:
import joblib

joblib.dump(ct, "SitePreprocessor.joblib")

['SitePreprocessor.joblib']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)

clf.fit(X_train, y_train)
print('Score: ',clf.score(X_test, y_test))

[22:48:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Score:  0.975


c:\Users\cvaal\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [ ]:
joblib.dump(clf, 'XGBoostClassifier.joblib')

['XGBoostClassifier.joblib']

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_absolute_error(y_test, y_pred)

0.025

In [ ]:
mean_squared_error(y_test, y_pred)


0.025

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

0.962962962962963

In [11]:
X_test

,Domain registrar,Postal code,Wordpress?,Page rank decimal,Site Rank,Country of origin,Malicious,No. of Local links,Total links
5,"GoDaddy.com, LLC",85284,True,4.79,31957.0,US,0,41,52
91,"TurnCommerce, Inc. DBA NameBright.com",Redacted for GDPR privacy,False,3.35,3302463.0,FR,1,49,51
0,"GoDaddy.com, LLC",85284,True,2.82,7390112.0,US,0,16,17
138,NOM-IQ Ltd dba Com Laude,02115-3012,False,5.82,2154.0,US,0,125,133
108,"Sssasss, Incorporated",Statutory Masking Enabled,False,2.21,19647496.0,GI,1,0,0
190,"Network Solutions, LLC",70810-1611,False,5.29,13773.0,US,0,138,177
94,"SAV.COM, LLC",926,False,2.54,10910748.0,NaN,1,0,0
36,"TUCOWS, INC.",REDACTED FOR PRIVACY,False,3.97,1327544.0,GB,0,45,69
170,"GoDaddy.com, LLC",85284,False,4.27,285251.0,US,0,125,155
141,"TUCOWS, INC.",REDACTED FOR PRIVACY,False,4.67,45840.0,US,0,174,181
